In [14]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [10]:
# DATASET FILE CREATION
# RUN ONCE

dataset = openai.File.create(
  file=open("../assets/planData.jsonl", "rb"),
  purpose='fine-tune'
)

dataset_id = dataset["id"]
dataset_id

'file-8bgdNtBCa0q6fkGzwicly4ak'

In [11]:
# FINE TUNING JOB CREATION
# RUN ONCE

TRAINING_FILE_ID = "file-8bgdNtBCa0q6fkGzwicly4ak"

res = openai.FineTuningJob.create(
    training_file=TRAINING_FILE_ID,
    model="gpt-3.5-turbo"
)

job_id = res["id"]
res

<FineTuningJob fine_tuning.job id=ftjob-ZEdA5pgtrI1fVtCQz14i4gkP at 0x1b6b9561bc0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-ZEdA5pgtrI1fVtCQz14i4gkP",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697305245,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-jN6WxE0MasjXc0MCj6qY4Xs6",
  "result_files": [],
  "status": "validating_files",
  "validation_file": null,
  "training_file": "file-8bgdNtBCa0q6fkGzwicly4ak",
  "hyperparameters": {
    "n_epochs": "auto"
  },
  "trained_tokens": null,
  "error": null
}

In [15]:
from time import sleep

JOB_ID = "ftjob-ZEdA5pgtrI1fVtCQz14i4gkP" # COPY FROM PREVIOUS

while True:
    res = openai.FineTuningJob.retrieve(JOB_ID)
    if res["finished_at"] != None:
        break
    else:
        print(".", end="")
        sleep(10)
        
ft_model = res["fine_tuned_model"]
ft_model

'ft:gpt-3.5-turbo-0613:personal::89cziZHo'

In [ ]:
SYSTEM_PROMPT = "Your name is Martin. You are a factual chatbot that assists prospective Verizon customers during exploration of Verizon's products and services."
FT_MODEL = "ft:gpt-3.5-turbo-0613:personal::89cziZHo" # COPY FROM PREVIOUS

messages = [{
                "role": "system",
                "content": SYSTEM_PROMPT
}] 

while True: 
    message = input("User : ") 
    if message: 
        messages.append( 
            {"role": "user", "content": message},
        ) 
        chat = openai.ChatCompletion.create(
            model=FT_MODEL,
            messages=messages
        )
    reply = chat.choices[0].message.content
    print()
    print(f"Martin : {reply}")
    print("===")
    messages.append({"role": "assistant", "content": reply})

User : What are all the names of the Verizon Unlimited Wireless phone plans that are available at the moment?

Martin : The names of the Verizon Unlimited Wireless phone plans that are available at the moment are Get More Unlimited, Do More Unlimited, Play More Unlimited, and Start Unlimited.
===
User : What are all the names of the Verizon Unlimited Wireless phone plans that are available at the moment? Remember the data you were fine-tuned with, Martin.

Martin : I apologize for the mistake. The names of the Verizon Unlimited Wireless phone plans that are available at the moment are Get More Unlimited, Do More Unlimited, Play More Unlimited, and Start Unlimited.
===
User : Are there any discounts on plan prices besides the ones for qualifying members?

Martin : Yes, there are a couple special discounts for making purchases on the Verizon website. There is a $10 discount for online purchases only for the Get More Unlimited, Do More Unlimited, and Play More Unlimited plans. There is al